## Formal Notation of a Search Problem

A search problem is defined as a **6-tuple**:

$$
P = \langle S,\; s_0,\; A,\; T,\; G,\; C \rangle
$$

Sometimes the cost function is omitted, but this is the full form.

---

### Symbols and Meanings

| Symbol          | Meaning                              |
| --------------- | ------------------------------------ |
| **S**           | Set of all possible states           |
| **s₀ ∈ S**      | Initial state                        |
| **A(s)**        | Set of actions possible in state _s_ |
| **T(s, a)**     | Transition function → next state     |
| **G(s)**        | Goal test (returns true or false)    |
| **C(s, a, s′)** | Step cost function                   |


### Breadth First Search


In [12]:
from collections import deque

In [10]:
# How deque works
dq = deque()

dq.append(2)
dq.append(3)
dq.append(4)
dq.append(5)

print(dq)

dq.appendleft(6)

print(dq)

x = dq.pop()

print(x)
print(dq)

deque([2, 3, 4, 5])
deque([6, 2, 3, 4, 5])
5
deque([6, 2, 3, 4])


In [22]:
# Spread operator and arguments of a method
cheap = ["apple", "banana", "guava"]
expensive = ["avacado", "cherry", "blueberry"]

mixture = [*cheap, *expensive]

print(cheap)
print(expensive)
print(mixture)

# JS => ...
# Python => *


def sum(*args):
    print(args)
    s = 0
    for num in args:
        s += num
    return s


print(sum(2, 3, 4, 5))

['apple', 'banana', 'guava']
['avacado', 'cherry', 'blueberry']
['apple', 'banana', 'guava', 'avacado', 'cherry', 'blueberry']
(2, 3, 4, 5)
14


In [28]:
# Inheritance
class NoElementError(Exception):
    def __init__(self, *args):
        super().__init__(*args)


class Queue:
    # constructor
    def __init__(self):
        self.array = []
        self.l = 0
        self.r = 0
        pass

    # push
    def push(self, x):
        self.array.append(x)
        self.r += 1

    # front
    def front(self):
        if self.empty():
            raise NoElementError("Queue is empty!")

        return self.array[self.l]

    # pop
    def pop(self):
        if self.empty():
            raise NoElementError("Queue is empty!")

        ans = self.array[self.l]
        self.l += 1

        return ans

    # empty
    def empty(self):
        return self.size() == 0

    # size
    def size(self):
        return self.r - self.l


queue = Queue()
queue.push(2)
queue.pop()

try:
    queue.front()
except NoElementError as err:
    print(err)
except Exception:
    pass

Queue is empty!


In [45]:
# Graph: Prebuilt or On-The-Fly

# Search problem:
# Start at number 2.
# In one move you can:
# multiply by 2
# add 1
# Goal: reach 9

s = 2
t = 9

q = deque()
vis = set()

# Add src to visited, also push into queue
q.append(s)
vis.add(s)

steps = 0

par = {s: None}

while len(q) != 0:
    n = len(q)
    found = False

    while n != 0:
        # extract front element
        curr = q.popleft()

        if curr == t:
            print(f"Target reached in {steps} steps!")
            found = True
            break

        next = []
        # Actions create new states from a single current state

        # Action 01: Add 1
        next.append(curr + 1)

        # Action 02: Multiply by 2
        next.append(curr * 2)

        for nxt in next:
            if nxt not in vis:
                vis.add(nxt)
                q.append(nxt)

                # mark the parent
                par[nxt] = curr

        n -= 1

    steps += 1

    if found:
        break

node = t

while node != None:
    if node == s:
        print(node)
    else:
        print(node, end=" <- ")
    node = par[node]

Target reached in 3 steps!
9 <- 8 <- 4 <- 2


In [65]:
DEPTH_LIMIT = 5
INF = 10**9


def dfs(curr: int, t: int, vis: set, par: dict, depth: int):
    if curr == t:
        return depth

    if curr > t:
        return INF

    if depth >= DEPTH_LIMIT:
        return INF

    best = INF

    for nxt in (curr + 1, curr * 2):
        if nxt not in vis:
            vis.add(nxt)
            par[nxt] = curr

            dist = dfs(nxt, t, vis, par, depth + 1)
            best = min(best, dist)

            # BACKTRACK
            vis.remove(nxt)

    return best


vis = {2}
par = {2: None}

print(f"Dist: {dfs(2, 9, vis, par, 0)}")

node = t
path = []
while node != None:
    path.append(node)
    node = par[node]


def print_path(path: list) -> None:
    n: int = len(path)
    for idx in range(n):
        if idx == n - 1:
            print(path[idx])
        else:
            print(path[idx], end=" --> ")


path.reverse()

print_path(path)

Dist: 3
2 --> 4 --> 8 --> 9


In [ ]:
class State:
    """
    Represents a node in the state space.
    Here, the state is simply an integer value.
    """

    def __init__(self, value: int):
        self.value = value

    def __eq__(self, other):
        return isinstance(other, State) and self.value == other.value

    def __ne__(self, other):
        return not isinstance(other, State) or self.value != other.value

    def __hash__(self):
        return hash(self.value)

    def __repr__(self):
        return f"State({self.value})"

    def __str__(self):
        return f"State({self.value})"

    def __lt__(self, other):
        return isinstance(other, State) and self.value < other.value

In [88]:
def successors(state: State) -> list[State]:
    """
    Generates all valid successor states from the given state.

    Actions:
    1. Add 1
    2. Multiply by 2

    Args:
        state (State): Current state

    Returns:
        list[State]: List of successor states
    """
    return [State(state.value + 1), State(state.value * 2)]

In [89]:
INFINITY = 10**9


def depth_limited_search(
    current: State, target: State, visited: set[State], depth: int, limit: int
) -> int:
    """
    Performs Depth-Limited Search (DLS).

    Args:
        current (State): Current node in search
        target (State): Goal state
        visited (set[State]): Set of visited states
        depth (int): Current depth in the search tree
        limit (int): Maximum depth allowed

    Returns:
        int: Minimum depth at which target is found, or INFINITY if not found
    """
    if current == target:
        return depth

    if depth >= limit:
        return INFINITY

    best_depth = INFINITY

    for child in successors(current):
        if child not in visited:
            visited.add(child)
            result = depth_limited_search(child, target, visited, depth + 1, limit)
            best_depth = min(best_depth, result)
            visited.remove(child)  # backtrack

    return best_depth

In [ ]:
def iterative_deepening_search(start: State, target: State, max_depth: int = 20) -> int:
    """
    Performs Iterative Deepening DFS (IDDFS).

    Args:
        start (State): Initial state
        target (State): Goal state
        max_depth (int): Maximum depth to try

    Returns:
        int: Depth at which target is found, or INFINITY
    """
    for limit in range(max_depth + 1):  # [1, max_depth]
        visited = {start}
        depth = depth_limited_search(start, target, visited, 0, limit)

        if depth != INFINITY:
            return depth

    return INFINITY

In [87]:
print(iterative_deepening_search(State(2), State(9), 10))

3


In [ ]:
import heapq


class PriorityQueue:
    def __init__(self):
        self.queue = []

    def push(self, item):
        heapq.heappush(self.queue, item)

    def pop(self):
        return heapq.heappop(self.queue)

    def top(self):
        return self.queue[0]

    def size(self):
        return len(self.queue)

    def empty(self):
        return self.size() == 0

In [100]:
import heapq as hq

# By default its min heap
q = []

print(q)

hq.heappush(q, 5)

print(q)

hq.heappush(q, 2)

print(q)

hq.heappush(q, 1)

print(q)

hq.heappush(q, -10)

print(q)

mini = hq.heappop(q)

print(mini)
print(q)

[]
[5]
[2, 5]
[1, 5, 2]
[-10, 1, 2, 5]
-10
[1, 5, 2]


In [111]:
def weighted_successors(state: State) -> list[(State, int)]:
    """
    Generates all valid successor states from the given state.

    Actions:
    1. Add 1
    2. Multiply by 2

    Args:
        state (State): Current state

    Returns:
        list[State]: List of successor states
    """
    return [(State(state.value + 1), 1), (State(state.value * 2), 1)]

In [ ]:
def uniform_cost_search(start: State, goal: State):
    pq = PriorityQueue()
    pq.push((0, start))

    cost = {start: 0}
    parent = {start: None}

    while not pq.empty():
        curr_cost, curr_state = pq.pop()

        if curr_state == goal:
            # reconstruct path
            path = []
            node = curr_state
            while node:
                path.append(node)
                node = parent[node]
            return curr_cost, path[::-1]

        for nxt, step_cost in weighted_successors(curr_state):
            new_cost = curr_cost + step_cost

            if nxt not in cost or new_cost < cost[nxt]:
                cost[nxt] = new_cost
                parent[nxt] = curr_state
                pq.push((new_cost, nxt))

    return None

In [118]:
cost, path = uniform_cost_search(State(2), State(9))
print(cost)
print(path)

3
[State(2), State(4), State(8), State(9)]


In [109]:
# Array slicing
arr = [1, 2, 3, 4, 5, 6, 7, 8]
print(arr)
slice = arr[:3]  # end is exclusive
print(slice)
slice = arr[:-1]
print(slice)
slice = arr[::]
print(slice)
slice = arr[::2]
print(slice)
slice = arr[2:6:2]
print(slice)
slice = arr[::-1]  # reverse copy
print(slice)

[1, 2, 3, 4, 5, 6, 7, 8]
[1, 2, 3]
[1, 2, 3, 4, 5, 6, 7]
[1, 2, 3, 4, 5, 6, 7, 8]
[1, 3, 5, 7]
[3, 5]
[8, 7, 6, 5, 4, 3, 2, 1]


In [ ]:
def build_path(meet, parent_f, parent_b):
    path_f = []
    n = meet
    while n:
        path_f.append(n)
        n = parent_f[n]
    path_f.reverse()

    path_b = []
    n = parent_b[meet]
    while n:
        path_b.append(n)
        n = parent_b[n]

    return path_f + path_b

In [124]:
# Bidirectional search
from collections import deque


def bidirectional_bfs(start, goal, successors, predecessors):
    if start == goal:
        return [start]

    q_f = deque([start])
    q_b = deque([goal])

    visited_f = {start}
    visited_b = {goal}

    parent_f = {start: None}
    parent_b = {goal: None}

    while q_f and q_b:

        # Forward step
        for _ in range(len(q_f)):
            curr = q_f.popleft()
            for nxt in successors(curr):
                if nxt not in visited_f:
                    visited_f.add(nxt)
                    parent_f[nxt] = curr
                    q_f.append(nxt)

                    if nxt in visited_b:
                        return build_path(nxt, parent_f, parent_b)

        # Backward step
        for _ in range(len(q_b)):
            curr = q_b.popleft()
            for prev in predecessors(curr):
                if prev not in visited_b:
                    visited_b.add(prev)
                    parent_b[prev] = curr
                    q_b.append(prev)

                    if prev in visited_f:
                        return build_path(prev, parent_f, parent_b)

    return None

In [125]:
def successors(state: State) -> list[State]:
    return [State(state.value + 1), State(state.value * 2)]


def predecessors(state: State) -> list[State]:
    ans = [State(state.value - 1)]

    if state.value % 2 == 0:
        ans.append(State(state.value / 2))
        
    return ans


print(
    bidirectional_bfs(
        State(2),
        State(9),
        successors,
        predecessors
    )
)

[State(2), State(4), State(8), State(9)]
